# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json`

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# copied the file to the working folder for ease of import that will run within the notebook
!mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json

2023-05-13T16:57:32.855-0300	connected to: mongodb://localhost/
2023-05-13T16:57:32.855-0300	dropping: uk_food.establishments
2023-05-13T16:57:34.410-0300	39779 document(s) imported successfully. 0 document(s) failed to import.


In [7]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'config', 'epa', 'local', 'met', 'uk_food']


In [8]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [9]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [10]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [11]:
# review a document in the establishments collection
pprint(db.establishments.find_one())

{'AddressLine1': 'The Pines Garden',
 'AddressLine2': 'Beach Road',
 'AddressLine3': 'St Margarets Bay',
 'AddressLine4': 'Kent',
 'BusinessName': 'The Pines Calyx',
 'BusinessType': 'Other catering premises',
 'BusinessTypeID': 7841,
 'ChangesByServerID': 0,
 'Distance': 4587.362402580997,
 'FHRSID': 254250,
 'LocalAuthorityBusinessID': 'PI/000066174',
 'LocalAuthorityCode': '182',
 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk',
 'LocalAuthorityName': 'Dover',
 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'CT15 6DZ',
 'RatingDate': '2021-08-17T00:00:00',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingValue': '5',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('645febac5ce28c6c52a4bf8d'),
 'geocode': {'latitude': '51.148133', 'longitude': '1.383298'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/254250',
            'rel': 'self'}],
 'meta': {'dataSource': None,
          'extract

In [14]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [15]:
# Create a dictionary for the new restaurant data
penang_flavours = {
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}

In [16]:
# Insert the new restaurant into the collection
establishments.insert_one(penang_flavours)

In [17]:
# Check that the new restaurant was inserted
db.establishments.find_one({'BusinessName' : 'Penang Flavours'})

{'_id': ObjectId('645febe4964f95c498ba926d'),
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '',
 'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'PostCode': 'SE18 7DY',
 'Phone': '',
 'LocalAuthorityCode': '511',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'scores': {'Hygiene': '', 'Structural': '', 'ConfidenceInManagement': ''},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '0.08384000', 'latitude': '51.49014200'},
 'RightToReply': '',
 'Distance': 4623.972328074718,
 'NewRatingPending': True}

2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [18]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {"BusinessType": "Restaurant/Cafe/Canteen"}
projection = {"BusinessTypeID": 1, "BusinessType": 1, "_id": 0}
results = db['establishments'].find(query, projection)
pprint(results[0])

{'BusinessType': 'Restaurant/Cafe/Canteen', 'BusinessTypeID': 1}


3. Update the new restaurant with the `BusinessTypeID` you found.

In [19]:
# Update the new restaurant with the correct BusinessTypeID. Using a variable method so it could be used again. 
filter = {"BusinessName": "Penang Flavours"}

# Define the update operation to set the BusinessTypeID to '1'
update = {"$set": {"BusinessTypeID": "1"}}

# Update the document in the collection
result = db['establishments'].update_one(filter, update)

In [30]:
# Confirm that the new restaurant was updated
Penang_Flavours = db.establishments.find_one({'BusinessName' : 'Penang Flavours'})

Penang_Flavours['BusinessTypeID']

'1'

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [31]:
# Find how many documents have LocalAuthorityName as "Dover"
filter = {"LocalAuthorityName": "Dover"}

# Count the number of documents that match the filter
count = db['establishments'].count_documents(filter)

# Print the count
print("Number of documents with LocalAuthorityName as Dover:", count)

Number of documents with LocalAuthorityName as Dover: 994


In [32]:
# Delete all documents where LocalAuthorityName is "Dover"
filter = {"LocalAuthorityName": "Dover"}

db['establishments'].delete_many(filter)

In [33]:
# Check if any remaining documents include Dover
filter = {"LocalAuthorityName": "Dover"}

# Count the number of documents that match the filter
count = db['establishments'].count_documents(filter)

# Print the count
print("Number of documents with LocalAuthorityName as Dover:", count)

Number of documents with LocalAuthorityName as Dover: 0


In [34]:
# Check that other documents remain with 'find_one'
db.establishments.find_one()

{'_id': ObjectId('645febac5ce28c6c52a4c274'),
 'FHRSID': 289352,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000002460',
 'BusinessName': 'The Ship',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'AddressLine1': '23 The Stade',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'PostCode': 'CT19 6AB',
 'Phone': '',
 'RatingValue': '5',
 'RatingKey': 'fhrs_5_en-gb',
 'RatingDate': '2015-08-27T00:00:00',
 'LocalAuthorityCode': '188',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 5},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '1.188537', 'latitude': '51.08084'},
 'RightToReply': '',
 'Distance': 4591.912144538092,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01T00:00:00',
  'itemCount': 0,

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [35]:
# Change the data type from String to Decimal for longitude and latitude
establishments.update_many({}, [{'$set': {'geocode.longitude': {'$toDouble': "$geocode.longitude"}}}])
establishments.update_many({}, [{'$set': {'geocode.latitude': {'$toDouble': "$geocode.latitude"}}}])

Use `update_many` to convert `RatingValue` to integer numbers.

In [36]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [37]:
# Change the data type from String to Integer for RatingValue
establishments.update_many({}, [{'$set': {'RatingValue': {'$toInt': "$RatingValue"}}}])

string_check = db.establishments.find_one()
print(type(string_check['RatingValue']))

<class 'int'>


In [38]:
# Check that the coordinates and rating value are now numbers
# Retrieve the document from the collection
num_check = db.establishments.find_one()

print(type(num_check['geocode']['longitude']))
print(type(num_check['geocode']['latitude']))

<class 'float'>
<class 'float'>
